In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
import lightgbm as lgb
import matplotlib.pyplot as plt
import pandas_datareader as pdr
import pickle
import time
import gc
from multiprocessing import *
from xgboost import XGBClassifier
from sklearn import *
from numba import jit
from sklearn.datasets import make_friedman1
from sklearn.svm import SVC, SVR
from sklearn.feature_selection import RFE, RFECV
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, Imputer, normalize
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Activation, Flatten, BatchNormalization, Dropout

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/learning_curve.py:22: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This module will be removed in 0.20
  DeprecationWarning)
Using TensorF

In [2]:
DATA_TRAIN_PATH = './Data/train.csv'
DATA_TEST_PATH =  './Data/test.csv'

def load_data(train_path = DATA_TRAIN_PATH, test_path = DATA_TEST_PATH):
    train = pd.read_csv(train_path, na_values = "-1", dtype = np.float64)
    train = train.fillna(-1)
    test = pd.read_csv(test_path, na_values = "-1")
    test = test.fillna(-1)
    
    x_train = train.drop(['target', 'id'], axis = 1)
    y_train = train['target']
    id_train = train['id'].values
    print('Train data shape: ', x_train.shape)
    
    x_test = test.drop(['id'], axis = 1)
    id_test = test['id'].values
    print('Test data shape: ', x_test.shape)   
    
    return x_train, y_train, id_train, x_test, id_test

def feature_info(df):
    cat_features = df.columns[df.columns.str.endswith('cat')].tolist()
    bin_features = df.columns[df.columns.str.endswith('bin')].tolist()
    num_features = [feature for feature in df.columns.tolist() 
                    if feature not in cat_features and feature not in bin_features]
    return cat_features, bin_features, num_features

def submit(filename, id_test, pred_test):
    sub = pd.DataFrame()
    sub['id'] = id_test
    sub['target'] = pred_test
    sub.to_csv(filename, index = False)

def gini(y_true, y_prob):
    y_true = np.asarray(y_true)
    y_true = y_true[np.argsort(y_prob)]
    ntrue = 0
    gini = 0
    delta = 0
    n = len(y_true)
    for i in range(n-1, -1, -1):
        y_i = y_true[i]
        ntrue += y_i
        gini += y_i * delta
        delta += 1 - y_i
    gini = 1 - 2 * gini / (ntrue * (n - ntrue))
    return gini

def gini_normalized(y_true, y_pred):
    return gini(y_true, y_pred)/gini(y_true, y_true)

def gini_xgb(preds, dtrain):
    labels = dtrain.get_label()
    gini_score = gini_normalized(labels, preds)
    return[('gini', gini_score)]

def gini_lgb(preds, dtrain):
    labels = dtrain.get_label()
    gini_score = gini_normalized(labels, preds)
    return 'gini', gini_score, True    
    
def add_noise(series, noise_level):
    return series*( 1+ noise_level * np.random.randn(len(series)))

def target_encoder(trn_series=None, tst_series=None, target=None, 
                    min_samples_leaf=1, smoothing=1, noise_level=0):
    
    """
    Smoothing is computed like in the following paper by Daniele Micci-Barreca
    https://kaggle2.blob.core.windows.net/forum-message-attachments/225952/7441/high%20cardinality%20categoricals.pdf
    trn_series : training categorical feature as a pd.Series
    tst_series : test categorical feature as a pd.Series
    target : target data as a pd.Series
    min_samples_leaf (int) : minimum samples to take category average into account
    smoothing (int) : smoothing effect to balance categorical average vs prior  
    """ 
        
    assert len(trn_series) == len(target)
    assert trn_series.name == tst_series.name
    temp = pd.concat([trn_series, target], axis=1)
    
    # Compute target mean 
    averages = temp.groupby(by=trn_series.name)[target.name].agg(["mean", "count"])
    
    # Compute smoothing
    smoothing = 1 / (1 + np.exp(-(averages["count"] - min_samples_leaf) / smoothing))
    
    # Apply average function to all target data
    prior = target.mean()
    
    # The bigger the count the less full_avg is taken into account
    averages[target.name] = prior * (1 - smoothing) + averages["mean"] * smoothing
    averages.drop(["mean", "count"], axis=1, inplace=True)
    
    # Apply averages to trn and tst series
    ft_trn_series = pd.merge(
        trn_series.to_frame(trn_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=trn_series.name,how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    
    # pd.merge does not keep the index so restore it    
    ft_trn_series.index = trn_series.index 
    
    ft_tst_series = pd.merge(
        tst_series.to_frame(tst_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=tst_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_tst_series.index = tst_series.index
    
    return add_noise(ft_trn_series, noise_level), add_noise(ft_tst_series, noise_level)   

def rfe_svc(x_train, y_train):
    
    print('Running RFE')
    estimator = SVC(kernel='rbf', C=1, verbose = True)        
    rfe = RFECV(estimator = estimator, step = 1, cv = 5 , verbose = 1)
    rfe = rfe.fit(x_train, y_train)    
    return rfe.ranking_, rfe.support_

def rfe_gb(x_train, y_train):
    
    print('Running RFE')
    estimator = ExtraTreesClassifier(n_estimators = 5, max_depth=5, min_samples_split=2, random_state=0, verbose=1)    
    estimator = estimator.fit(x_train, y_train)
    rfe = RFECV(estimator = estimator, step = 1, cv = 5 , verbose = 1)
    rfe = rfe.fit(x_train, y_train) 
    
    return rfe

def transform_df(df_train, df_test, df_label):             
    
    df_train = pd.DataFrame(df_train)   
    df_test = pd.DataFrame(df_test)
    
    cat_features, bin_features, num_features = feature_info(df_train)
    
#     for k in bin_features:
#         print(k)
    
    for i in cat_features:       
        imp = Imputer(missing_values = -1, strategy = 'most_frequent', axis = 0, verbose =1, copy = False)       
        df_train[i] = imp.fit_transform(df_train[i].reshape(-1,1))        
        df_test[i] = imp.fit_transform(df_test[i].reshape(-1,1))        
        
        df_train[i], df_test[i] = target_encoder(df_train[i], df_test[i], df_label, 
                                                min_samples_leaf = 100, smoothing = 10,
                                               noise_level = 0.010)    
    
    
#     for i in num_features:
#         df_train[i] = normalize(df_train[i].reshape(-1,1))
#         df_test[i] = normalize(df_test[i].reshape(-1,1))
    
    df_train = df_train.drop(['ps_ind_10_bin', 'ps_ind_11_bin', 'ps_ind_12_bin', 'ps_ind_13_bin'], axis = 1)
    df_train['ps_car_13_x_ps_reg_03'] = df_train['ps_car_13'] * df_train['ps_reg_03']    
    df_train['ps_car_02_cat_x_ps_reg_01'] = df_train['ps_car_02_cat'] * df_train['ps_reg_01']
    df_train['ps_car_04_cat_x_ps_reg_01'] = df_train['ps_car_04_cat'] * df_train['ps_reg_01']

    df_test = df_test.drop(['ps_ind_10_bin', 'ps_ind_11_bin', 'ps_ind_12_bin', 'ps_ind_13_bin'], axis = 1)
    df_test['ps_car_13_x_ps_reg_03'] = df_test['ps_car_13'] * df_test['ps_reg_03']
    df_test['ps_car_02_cat_x_ps_reg_01'] = df_test['ps_car_02_cat'] * df_test['ps_reg_01']
    df_test['ps_car_04_cat_x_ps_reg_01'] = df_test['ps_car_04_cat'] * df_test['ps_reg_01']            
    
    
    print('Train data shape: ', df_train.shape)
    print('Test data shape: ', df_test.shape)
    
    return df_train, df_test   

In [3]:
x_train, y_train, id_train, x_test, id_test = load_data(train_path = DATA_TRAIN_PATH, test_path = DATA_TEST_PATH)
x_train, x_test=transform_df(x_train, x_test, y_train)

# rfe_svc = rfe_svc(x_train, y_train)
# print(rfe_svc.ranking_)
# print(rfe_svc.support_)
# rfe_gb = rfe_gb(x_train, y_train)
# print(rfe_gb.ranking_)
# print(rfe_gb.support_)

Train data shape:  (595212, 57)
Test data shape:  (892816, 57)
Train data shape:  (595212, 56)
Test data shape:  (892816, 56)


In [4]:
# ranking, support = rfe_svc(x_train, y_train)

In [5]:
# rfe_model = rfe_gb(x_train, y_train)
# x_train = pd.DataFrame(rfe_model.transform(x_train))
# x_test =  pd.DataFrame(rfe_model.transform(x_test))
# print(x_train.shape, x_test.shape)

In [6]:
kfold = 5
num_round = 10000
early_stopping_rounds = 200

In [7]:
lgb_params = {}

# Decent
lgb_params[0] = {}
lgb_params[0]['boosting'] = 'gbdt'
lgb_params[0]['learning_rate'] = 1e-2
lgb_params[0]['max_depth'] = 6
#lgb_params[0]['num_leaves'] = int(0.90 * (2**lgb_params[0]['max_depth']))
lgb_params[0]['max_bin'] = 15
lgb_params[0]['feature_fraction'] = 0.8
lgb_params[0]['bagging_fraction'] = 0.8
lgb_params[0]['bagging_freq'] = 5
lgb_params[0]['min_data'] = 500
lgb_params[0]['objective'] = 'binary'
lgb_params[0]['metric'] = 'auc'
lgb_params[0]['lambda_l1'] = 0.00000
lgb_params[0]['lambda_l2'] = 0.00000
lgb_params[0]['min_sum_hessian_in_leaf'] = 1e-3
lgb_params[0]['min_gain_to_split'] = 0.00000
lgb_params[0]['bagging_seed'] = 99

# Decent
lgb_params[1] = {}
lgb_params[1]['boosting'] = 'gbdt'
lgb_params[1]['learning_rate'] = 5e-3
lgb_params[1]['max_depth'] = 5
lgb_params[1]['max_bin'] = 10
lgb_params[1]['feature_fraction'] = 0.8
lgb_params[1]['bagging_fraction'] = 0.9
lgb_params[1]['bagging_freq'] = 5
lgb_params[1]['min_data'] = 500
lgb_params[1]['objective'] = 'binary'
lgb_params[1]['metric'] = 'auc'
lgb_params[1]['lambda_l1'] = 5
lgb_params[1]['lambda_l2'] = 9
lgb_params[1]['min_sum_hessian_in_leaf'] = 1e-3
lgb_params[1]['min_gain_to_split'] = 0.00000
lgb_params[1]['bagging_seed'] = 99

lgb_params[2] = {}
lgb_params[2]['boosting'] = 'gbdt'
lgb_params[2]['learning_rate'] = 0.02
lgb_params[2]['max_depth'] = 5
lgb_params[2]['num_leaves'] = int(0.70 * (2**lgb_params[2]['max_depth']))
lgb_params[2]['max_bin'] = 10
lgb_params[2]['feature_fraction'] = 0.8
lgb_params[2]['bagging_fraction'] = 0.9
lgb_params[2]['bagging_freq'] = 5
lgb_params[2]['min_data'] = 500
lgb_params[2]['objective'] = 'binary'
lgb_params[2]['metric'] = 'auc'
lgb_params[2]['lambda_l1'] = 3
lgb_params[2]['lambda_l2'] = 7
lgb_params[2]['min_sum_hessian_in_leaf'] = 1e-3
lgb_params[2]['min_gain_to_split'] = 0.00000
lgb_params[2]['bagging_seed'] = 99

lgb_params[3] = {}
lgb_params[3]['boosting'] = 'gbdt'
lgb_params[3]['learning_rate'] = 5e-3
lgb_params[3]['max_depth'] = 5
lgb_params[3]['num_leaves'] = int(0.80 * (2**lgb_params[3]['max_depth']))
lgb_params[3]['max_bin'] = 10
lgb_params[3]['feature_fraction'] = 0.8
lgb_params[3]['bagging_fraction'] = 0.9
lgb_params[3]['bagging_freq'] = 5
lgb_params[3]['min_data'] = 500
lgb_params[3]['objective'] = 'binary'
lgb_params[3]['metric'] = 'auc'
lgb_params[3]['lambda_l1'] = 0.50000
lgb_params[3]['lambda_l2'] = 0.50000
lgb_params[3]['min_sum_hessian_in_leaf'] = 1e-3
lgb_params[3]['min_gain_to_split'] = 0.00000
lgb_params[3]['bagging_seed'] = 99

lgb_params[4] = {}
lgb_params[4]['boosting'] = 'gbdt'
lgb_params[4]['learning_rate'] = 5e-3
lgb_params[4]['max_depth'] = 5
lgb_params[4]['num_leaves'] = int(0.70 * (2**lgb_params[4]['max_depth']))
lgb_params[4]['max_bin'] = 10
lgb_params[4]['feature_fraction'] = 0.8
lgb_params[4]['bagging_fraction'] = 0.9
lgb_params[4]['bagging_freq'] = 5
lgb_params[4]['min_data'] = 500
lgb_params[4]['objective'] = 'binary'
lgb_params[4]['metric'] = 'auc'
lgb_params[4]['lambda_l1'] = 0.00000
lgb_params[4]['lambda_l2'] = 0.00000
lgb_params[4]['min_sum_hessian_in_leaf'] = 1e-3
lgb_params[4]['min_gain_to_split'] = 0.00000
lgb_params[4]['bagging_seed'] = 99

lgb_params[5] = {}
lgb_params[5]['learning_rate'] = 0.02
lgb_params[5]['n_estimators'] = 650
lgb_params[5]['max_bin'] = 10
lgb_params[5]['subsample'] = 0.8
lgb_params[5]['subsample_freq'] = 10  
lgb_params[5]['min_child_samples'] = 500
lgb_params[5]['feature_fraction'] = 0.9
lgb_params[5]['bagging_freq'] = 1
lgb_params[5]['objective'] = 'binary'
lgb_params[5]['metric'] = 'auc'
lgb_params[5]['seed'] = 200

lgb_params[6] = {}
lgb_params[6]['n_estimators'] = 1090
lgb_params[6]['learning_rate'] = 0.02   
lgb_params[6]['subsample'] = 0.7
lgb_params[6]['subsample_freq'] = 2
lgb_params[6]['num_leaves'] = 16
lgb_params[6]['feature_fraction'] = 0.8
lgb_params[6]['bagging_freq'] = 1
lgb_params[6]['objective'] = 'binary'
lgb_params[6]['metric'] = 'auc'
lgb_params[6]['seed'] = 200


lgb_params[7] = {}
lgb_params[7]['n_estimators'] = 1100
lgb_params[7]['max_depth'] = 4
lgb_params[7]['learning_rate'] = 0.02
lgb_params[7]['feature_fraction'] = 0.95
lgb_params[7]['bagging_freq'] = 1
lgb_params[7]['objective'] = 'binary'
lgb_params[7]['metric'] = 'auc'
lgb_params[7]['seed'] = 200


In [8]:
y_pred = np.zeros((x_test.shape[0],1))

for k in range(7):
    
    print('Parameters set: '+ str(k))
    stratkfold = StratifiedKFold(n_splits=kfold, random_state=None, shuffle = True)
    
    for i, (train_index, test_index) in enumerate(stratkfold.split(x_train, y_train)):   
        print(' light gbm kfold: {}  of  {} : '.format(i+1, kfold))    
        d_train = lgb.Dataset(x_train.iloc[train_index], label = y_train.iloc[train_index]) 
        d_valid = lgb.Dataset(x_train.iloc[test_index], label = y_train.iloc[test_index]) 

        model = lgb.train(lgb_params[k], d_train, num_round, d_valid, early_stopping_rounds = early_stopping_rounds, 
                      feval = gini_lgb, verbose_eval = 200)
        #model = lgb.train(lgb_params[k], d_train, num_round, d_valid, early_stopping_rounds = early_stopping_rounds, 
#                       feval = None, verbose_eval = 200)
        y_pred += np.reshape(model.predict(x_test, num_iteration=(model.best_iteration)), (-1,1))
        print("_"*90 + '\n')

    model.save_model('./Model/lgb_model_'+ str(k) + '.txt')
    print("*"*90)
    print("*"*90 + '\n')
    
y_pred_lgb = np.array(y_pred) / (7*kfold)

Parameters set: 0
 light gbm kfold: 1  of  5 : 
Training until validation scores don't improve for 200 rounds.
[200]	valid_0's auc: 0.622635	valid_0's gini: 0.245271
[400]	valid_0's auc: 0.631241	valid_0's gini: 0.262482
[600]	valid_0's auc: 0.637368	valid_0's gini: 0.274736
[800]	valid_0's auc: 0.640254	valid_0's gini: 0.280508
[1000]	valid_0's auc: 0.641301	valid_0's gini: 0.282602
[1200]	valid_0's auc: 0.641783	valid_0's gini: 0.283566
[1400]	valid_0's auc: 0.641711	valid_0's gini: 0.283422
Early stopping, best iteration is:
[1245]	valid_0's auc: 0.642051	valid_0's gini: 0.284101
__________________________________________________________________________________________

 light gbm kfold: 2  of  5 : 
Training until validation scores don't improve for 200 rounds.
[200]	valid_0's auc: 0.619527	valid_0's gini: 0.239054
[400]	valid_0's auc: 0.62582	valid_0's gini: 0.25164
[600]	valid_0's auc: 0.629958	valid_0's gini: 0.259916
[800]	valid_0's auc: 0.631881	valid_0's gini: 0.263762
[1000]	

 light gbm kfold: 1  of  5 : 
Training until validation scores don't improve for 200 rounds.
[200]	valid_0's auc: 0.641482	valid_0's gini: 0.282965
[400]	valid_0's auc: 0.646673	valid_0's gini: 0.293346
[600]	valid_0's auc: 0.647162	valid_0's gini: 0.294323
Early stopping, best iteration is:
[513]	valid_0's auc: 0.647599	valid_0's gini: 0.295198
__________________________________________________________________________________________

 light gbm kfold: 2  of  5 : 
Training until validation scores don't improve for 200 rounds.
[200]	valid_0's auc: 0.630293	valid_0's gini: 0.260586
[400]	valid_0's auc: 0.637325	valid_0's gini: 0.27465
[600]	valid_0's auc: 0.638445	valid_0's gini: 0.27689
[800]	valid_0's auc: 0.638316	valid_0's gini: 0.276632
Early stopping, best iteration is:
[737]	valid_0's auc: 0.638551	valid_0's gini: 0.277101
__________________________________________________________________________________________

 light gbm kfold: 3  of  5 : 
Training until validation scores don'

Training until validation scores don't improve for 200 rounds.
[200]	valid_0's auc: 0.626339	valid_0's gini: 0.252677
[400]	valid_0's auc: 0.628681	valid_0's gini: 0.257361
[600]	valid_0's auc: 0.633136	valid_0's gini: 0.266271
[800]	valid_0's auc: 0.63769	valid_0's gini: 0.27538
[1000]	valid_0's auc: 0.640659	valid_0's gini: 0.281317
[1200]	valid_0's auc: 0.642702	valid_0's gini: 0.285403
[1400]	valid_0's auc: 0.643958	valid_0's gini: 0.287916
[1600]	valid_0's auc: 0.644859	valid_0's gini: 0.289718
[1800]	valid_0's auc: 0.64542	valid_0's gini: 0.290839
[2000]	valid_0's auc: 0.645567	valid_0's gini: 0.291134
[2200]	valid_0's auc: 0.645625	valid_0's gini: 0.291251
[2400]	valid_0's auc: 0.645668	valid_0's gini: 0.291336
[2600]	valid_0's auc: 0.645575	valid_0's gini: 0.291151
Early stopping, best iteration is:
[2506]	valid_0's auc: 0.64571	valid_0's gini: 0.291421
__________________________________________________________________________________________

 light gbm kfold: 3  of  5 : 
Trai

[200]	valid_0's auc: 0.626854	valid_0's gini: 0.253709
[400]	valid_0's auc: 0.634826	valid_0's gini: 0.269652
[600]	valid_0's auc: 0.635323	valid_0's gini: 0.270646
[800]	valid_0's auc: 0.635285	valid_0's gini: 0.270571
Early stopping, best iteration is:
[690]	valid_0's auc: 0.635781	valid_0's gini: 0.271562
__________________________________________________________________________________________

 light gbm kfold: 5  of  5 : 
Training until validation scores don't improve for 200 rounds.
[200]	valid_0's auc: 0.625798	valid_0's gini: 0.251596
[400]	valid_0's auc: 0.633828	valid_0's gini: 0.267655
[600]	valid_0's auc: 0.635156	valid_0's gini: 0.270311
[800]	valid_0's auc: 0.635422	valid_0's gini: 0.270845
Early stopping, best iteration is:
[685]	valid_0's auc: 0.635679	valid_0's gini: 0.271358
__________________________________________________________________________________________

******************************************************************************************
************

In [9]:
xgb_params = {}

#OK
xgb_params[0] = {}
xgb_params[0]['eta'] = 0.04 
xgb_params[0]['max_depth'] = 6
xgb_params[0]['subsample'] = 0.8
xgb_params[0]['colsample_bytree'] = 0.8
xgb_params[0]['objective'] = 'binary:logistic'
xgb_params[0]['eval_metric'] = 'logloss'
xgb_params[0]['seed'] = 99
xgb_params[0]['silent'] = True
xgb_params[0]['min_child_weight'] =  9.15
xgb_params[0]['lambda'] =  4.8
xgb_params[0]['alpha'] =  10.4
xgb_params[0]['gamma'] = 0.00

#GOOD
xgb_params[1] = {}
xgb_params[1]['eta'] = 0.010
xgb_params[1]['max_depth'] = 5
xgb_params[1]['subsample'] = 0.8
xgb_params[1]['colsample_bytree'] = 0.8
xgb_params[1]['objective'] = 'binary:logistic'
xgb_params[1]['eval_metric'] = 'logloss'
xgb_params[1]['seed'] = 99
xgb_params[1]['silent'] = True
xgb_params[1]['min_child_weight'] =  6
xgb_params[1]['lambda'] =  5
xgb_params[1]['alpha'] =  10
xgb_params[1]['gamma'] = 0.60

# GOOD
xgb_params[2] = {}
xgb_params[2]['eta'] = 0.05
xgb_params[2]['max_depth'] = 6
xgb_params[2]['subsample'] = 0.8
xgb_params[2]['colsample_bytree'] = 0.8
xgb_params[2]['objective'] = 'binary:logistic'
xgb_params[2]['eval_metric'] = 'auc'
xgb_params[2]['seed'] = 99
xgb_params[2]['silent'] = True
xgb_params[2]['lambda'] =  0.0000
xgb_params[2]['alpha'] =  0.0000
xgb_params[2]['gamma'] = 0.0000

In [10]:
# y_pred = np.zeros((x_test.shape[0],1))

# for k in range(3):
#     print('Parameters set: '+ str(k))
#     stratkfold = StratifiedKFold(n_splits=kfold, random_state=None, shuffle = True)
    
#     for i, (train_index, test_index) in enumerate(stratkfold.split(x_train, y_train)):
#         print(' xgb kfold: {}  of  {} : '.format(i+1, kfold))    
#         d_train = xgb.DMatrix(x_train.iloc[train_index], y_train.iloc[train_index]) 
#         d_valid = xgb.DMatrix(x_train.iloc[test_index], y_train.iloc[test_index]) 
#         watchlist = [(d_train, 'train'), (d_valid, 'valid')]

#         model = xgb.train(xgb_params[k], d_train, num_round, watchlist, early_stopping_rounds = early_stopping_rounds, 
#                       feval = gini_xgb, maximize = True, verbose_eval = 50)

#         y_pred += np.reshape(model.predict(xgb.DMatrix(x_test), ntree_limit=model.best_ntree_limit), (-1,1))
#         print("_"*90 + '\n')
        
#     pickle.dump(model, open("./Model/xgb_model_" + str(k) + ".pickle.dat", "wb")) 
#     print("*"*90)
#     print("*"*90 + '\n')
# y_pred_xgb = np.array(y_pred) / (3*kfold)

In [11]:
y_final = y_pred_lgb
submit('gb_mean_1.csv', id_test, y_final)

In [12]:
# y_final = 0.7*y_pred_lgb + 0.3*y_pred_xgb
# submit('gb_mean_1.csv', id_test, y_final)

In [13]:
# y_final = 0.5*y_pred_lgb + 0.5*y_pred_xgb
# submit('gb_mean_2.csv', id_test, y_final)

In [14]:
# y_final = 1*y_pred_lgb + 0*y_pred_xgb
# submit('gb_mean_4.csv', id_test, y_final)

In [15]:
x_train = x_train.as_matrix()
y_train = y_train.as_matrix()

In [16]:
def create_model():
    model = Sequential()
    model.add(Dense(256, input_dim = 56,
                    kernel_initializer = 'glorot_normal',
                    activation = 'relu'))
    model.add(Dropout(0.60))
    model.add(Dense(256, activation = 'relu'))
    model.add(Dropout(0.60))
    model.add(Dense(256, activation = 'relu'))
    model.add(Dropout(0.60))
    model.add(Dense(256, kernel_initializer = 'glorot_normal',
                    activation = 'relu'))
    model.add(Dropout(0.60))
    model.add(Dense(1, kernel_initializer = 'glorot_normal',
                    activation = 'softmax'))
    model.compile(optimizer = 'Adam', 
                  loss = 'mse',
                  metrics = ['accuracy'],
                 )
    return model

def train_model(model, x_train, y_train):
    history = model.fit(x = x_train, y = y_train, batch_size = 128, shuffle = True, 
              epochs = 100, verbose = 1, validation_split = 0.4, class_weight={0: 0.04, 1: 5},)
    return history

def _nn_predict(model, x_test):
    return model.predict(x_test)


In [17]:
# nn_model = create_model()
# nn_model = train_model(nn_model, x_train, y_train)
# nn_predict = nn_predict(nn_model, x_test)

